In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()



True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [2]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",temperature=0.9,max_tokens=500
)

@tool
def SqlQueryWritter(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
   
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent to write the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content



@tool
def SqlQuerySummarizer(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent , Summarize the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content


In [3]:
tools=[SqlQueryWritter,SqlQuerySummarizer]



In [25]:
# Define the tools
#tools = [SqlQueryWritter]
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
# Create the prompt for the agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant to write queries or summarize queries"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


In [8]:
#from langchain.agents.initialize import initialize_agent
#from langchain.agents import create_react_agent


from langchain.agents import load_tools                                         
from langchain.agents import initialize_agent
# Create the agent using the tools
#agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
chat_history = []

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt 
)


In [26]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
llm_withtools=llm.bind_tools(tools)
agent=(
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_withtools
    | OpenAIToolsAgentOutputParser()
)

In [27]:
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferWindowMemory
#memory = ConversationBufferWindowMemory(memory_key="chat_history",input_key="input",output_key='output', return_messages=True, k=4)



agent_executor = AgentExecutor(agent=agent, tools=tools)
                               

In [29]:
res=agent_executor.invoke({"input":"top 20 records sql sql query"})

In [32]:
print(res["output"])

To retrieve the top 20 records from a database table using SQL, you can use different SQL syntax depending on the database you are using. Here are some examples:

### For MySQL, PostgreSQL, SQLite
```sql
SELECT *
FROM your_table_name
ORDER BY some_column
LIMIT 20;
```

### For SQL Server
```sql
SELECT TOP 20 *
FROM your_table_name
ORDER BY some_column;
```

### For Oracle
Oracle doesn't use `LIMIT` or `TOP`. Instead, you can use:

#### Using `FETCH` Clause
```sql
SELECT *
FROM your_table_name
ORDER BY some_column
FETCH FIRST 20 ROWS ONLY;
```

#### Using `ROWNUM`
```sql
SELECT *
FROM (
  SELECT *
  FROM your_table_name
  ORDER BY some_column
)
WHERE ROWNUM <= 20;
```

Ensure to replace `your_table_name` with the actual table name and `some_column` with the column you wish to order the records by. The `ORDER BY` clause is crucial to define which records are considered the "top" ones.
